In [8]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np

In [9]:
from EWC_model import *
from train import * 
from training_environment_setup import get_all_training_environments

In [10]:
# -conv-layers
'''
        depth=depth,
        conv_type=args.conv_type if depth>0 else None,
        start_channels=args.channels if depth>0 else None,
        reducing_layers=args.rl if depth>0 else None,
        num_blocks=args.n_blocks if depth>0 else None,
        conv_bn=(True if args.conv_bn=="yes" else False) if depth>0 else None,
        conv_nl=args.conv_nl if depth>0 else None,
        no_fnl=True if depth>0 else None,
        global_pooling=checkattr(args, 'gp') if depth>0 else None,
'''
model = Classifier(
        image_size=32*32,
        image_channels=1,
        classes=10,
        # -fc-layers
        fc_units=1000,
        fc_drop=0.2,
        fc_layers = 4,
        fc_bn=True

    )

model

AttributeError: module 'torch' has no attribute '_six'

In [20]:
model.importance_weighting=='fisher'
model.precondition = True
model.fisher_n = None
model.fisher_labels = 'all'
model.fisher_batch = 1
# -options relating to 'Offline EWC' (Kirkpatrick et al., 2017) and 'Online EWC' (Schwarz et al., 2018)
model.offline = True
model.weight_penalty = False
model.reg_strength = 500

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else :
    device = torch.device('cpu')
model.to(device)

In [21]:
train_lb, train_powerlaw, train_ub , test = 

context:   0%|          | 0/10 [00:00<?, ?it/s]

1


accuracy for iteration  0  : 0.8421672952586207


accuracy for iteration  100  : 0.973700161637931


accuracy for iteration  200  : 0.9928609913793103


...

accuracy for iteration  0  : 0.15712553879310345


accuracy for iteration  100  : 0.4756869612068966


accuracy for iteration  200  : 0.5791352370689655


context:  50%|█████     | 5/10 [16:52<16:52, 202.51s/it]


KeyboardInterrupt: 

In [ ]:
for i, test_set in enumerate(tqdm(test_permutations)):
    test_loader = DataLoader(test_set,batch_size=256, shuffle=True)
    acc = []
    for batch_idx, (data, y) in enumerate(tqdm(test_loader)):
            # Perform training-step on this batch
            y_hat = model(data)
            acc.append((y == y_hat.max(1)[1]).sum().item() / data.size(0))
    print("Accuracy for context ", i , "is : ", np.mean(acc))

NameError: name 'tqdm' is not defined

In [7]:
OVERALL_DATA_SIZE = 60000

In [8]:
for task in range(10):
  perm_x_train = np.empty((0,32,32))
  perm_y_train = np.empty((0,))
  tensor_arr = np.array(list(training_permutations[0]))[:,0]
  # perm_y_train = np.array(list(training_permutations[0]))[:,1]
  for idx in range(OVERALL_DATA_SIZE):
    perm_x_train = np.vstack((perm_x_train, np.array(list(training_permutations[task]))[idx,0].numpy()))
    # perm_x_train = np.vstack((perm_x_train, tensor_arr[idx].numpy()))
    perm_y_train = np.hstack((perm_y_train, training_permutations[task][idx][1]))
  np.save('permuted_dataset_task_' + str(task) + '_X.npy', perm_x_train, allow_pickle=True)
  np.save('permuted_dataset_task_' + str(task) + '_Y.npy', perm_y_train, allow_pickle=True)

/var/folders/6_/w6mgwpfd12z46rvjfl18xd400000gn/T/ipykernel_16057/842474794.py:4: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  tensor_arr = np.array(list(training_permutations[0]))[:,0]
/var/folders/6_/w6mgwpfd12z46rvjfl18xd400000gn/T/ipykernel_16057/842474794.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor_arr = np.array(list(training_permutations[0]))[:

IndexError: index 1000 is out of bounds for axis 0 with size 1000